[![Notebook Tutorial](__code/__all/notebook_tutorial.png)](https://neutronimaging.pages.ornl.gov/en/tutorial/notebooks/normalization)

# Select Your IPTS 

In [1]:
from __code.normalization_new_ui import *

from __code.ui_builder import UiBuilder
o_builder = UiBuilder(ui_name = 'ui_roi_selection.ui')
from __code.roi_selection_ui import Interface

from __code import system
system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

pyuic5 /Users/j35/git/IPTS/python_notebooks/ui/ui_roi_selection.ui -o /Users/j35/git/IPTS/python_notebooks/__code/ui_roi_selection.py
** Using Debugging Mode! **


VBox(children=(HBox(children=(Label(value='IPTS-'), Text(value='', layout=Layout(width='10%')), Label(value='DOES NOT EXIST!', layout=Layout(width='20%'), _dom_classes=('result_label',)))), Label(value='OR'), HBox(children=(Label(value='Select Folder', layout=Layout(width='20%')), Select(layout=Layout(height='300px'), options=('BraggEdge', 'Brianne', 'DOE_letters', 'FITSimages', 'IPTS-12534', 'IPTS-13639_resonance_shawn', 'IPTS-14832', 'IPTS-15177', 'IPTS-16212', 'IPTS-16259', 'IPTS-16475', 'IPTS-16893', 'IPTS-17099', 'IPTS-17685', 'IPTS-17981', 'IPTS-18580', 'IPTS-18806', 'IPTS-18813', 'IPTS-19037', 'IPTS-19558', 'IPTS-19621', 'IPTS-19626', 'IPTS-20209-TOPAZ', 'IPTS_test_normalization', 'IPTS_testing_time_spectra', 'ImagesToRegisterForGian', 'Indu', 'Keita_code', 'Pour_ibeatles_video', 'ResonanceImaging', 'SENSITIVE', 'SarmaGUI', 'SphereCoordinates3D', 'TempData', 'gian', 'iBeatles_debugging', 'shared'), value='BraggEdge'))), Button(button_style='info', description='HELP', style=ButtonStyle())))

# Description 

This notebook performs the normalization of a set of images (tiff or fits) by OBs. 

The steps are:
 - select your **sample images**
 - select your **ob images**
 - select your **df images** (optional)
 - all the data will be loaded
 - preview of the sample and ob
 - optionally, selection of one or more **background** region in the sample data
 - normalization is performed
 - export of the normalized data

# Python Import

In [2]:
%gui qt

In [3]:
%matplotlib notebook

# Loading Images 

In [4]:
files = Files()
sample_panel = SampleSelectionPanel(working_dir=system.System.get_working_dir())
sample_panel.init_ui(files=files)
wizard = WizardPanel(sample_panel=sample_panel)

HTML(value='Please wait...')

Label(value='Selection of All Input Files', layout=Layout(border='1px lighgray solide', height='35px', padding='8px', width='300px'))

VBox(children=(HBox(children=(Label(value='Instructions:', layout=Layout(width='20%')), Label(value='Select list of Sample files (mandatory) ', layout=Layout(width='50%')))), HBox(children=(Label(value='Sample selected:', layout=Layout(width='20%')), Label(value='0 files selected', layout=Layout(width='50%'))))), layout=Layout(border='1px lightgray solid', margin='5px', padding='15px'))

HBox(children=(Label(value='         ', layout=Layout(width='70%')), Button(button_style='warning', description='Next Step>>', disabled=True, layout=Layout(width='20%'), style=ButtonStyle(), tooltip='Click to move to next step')))

VBox(children=(Label(value='', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-16259', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' .DS_Store                                         |     Mon Feb 12 12:59:05 2018', ' Day_4_CO-8_Uptake                                 |     Directory', ' Day_4_CO-8_Uptake_profiles                        |     Directory', ' Day_4_CO-8_Uptake_profiles.txt                    |     Fri Dec  8 12:58:35 2017', ' Day_4_CO-8_Uptake_profiles_DEBUGGING              |     Directory', ' Day_4_CO-8_Uptake_timestamp_infos.txt             |     Wed Jan 10 15:30:58 2018', ' MS-19                                             |     Directory', ' OB                                                |     Directory', ' df                                                |     Directory', ' from_analysis_computer                            |     Directory', ' ideal_water_intake_images                         |     Directory', ' ideal_water_intake_images_profiles                |     Directory', ' ideal_water_intake_images_profiles.txt            |     Wed Dec 13 10:23:47 2017', ' ideal_water_intake_images_timestamp_infos.txt     |     Wed Dec 13 10:21:31 2017'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

...

HTML(value='Please wait...')

VBox(children=(Label(value='', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-16259/df', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' 20171027_DF_0000_0020.tiff     |     Fri Oct 27 12:42:50 2017', ' 20171027_DF_0000_0021.tiff     |     Fri Oct 27 12:42:50 2017', ' 20171027_DF_0000_0022.tiff     |     Fri Oct 27 12:42:50 2017', ' 20171027_DF_0000_0023.tiff     |     Fri Oct 27 12:42:50 2017', ' 20171027_DF_0000_0024.tiff     |     Fri Oct 27 12:42:50 2017', ' 20171027_DF_0000_0025.tiff     |     Fri Oct 27 12:42:50 2017', ' 20171027_DF_0000_0026.tiff     |     Fri Oct 27 12:42:50 2017', ' 20171027_DF_0000_0027.tiff     |     Fri Oct 27 12:42:50 2017', ' 20171027_DF_0000_0028.tiff     |     Fri Oct 27 12:42:50 2017', ' 20171027_DF_0000_0029.tiff     |     Fri Oct 27 12:42:50 2017'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

HBox(children=(Label(value='Loading sample', layout=Layout(width='20%')), IntProgress(value=0, max=4)))

HBox(children=(Label(value='Loading ob', layout=Layout(width='20%')), IntProgress(value=0, max=5)))

HBox(children=(Label(value='Loading df', layout=Layout(width='20%')), IntProgress(value=0, max=3)))

# Preview Data

## sample

In [5]:
o_norm = sample_panel.o_norm
o_norm.plot_images(data_type='sample')

interactive(children=(IntSlider(value=0, continuous_update=False, description='sample Index', max=3), Output()), _dom_classes=('widget-interact',))

## ob

In [6]:
o_norm.plot_images(data_type='ob')

interactive(children=(IntSlider(value=0, continuous_update=False, description='ob Index', max=4), Output()), _dom_classes=('widget-interact',))

# Select Background Region 

In [7]:
o_gui = Interface(o_norm=o_norm.o_norm)
o_gui.show()

# Recap. 

In [8]:
o_gui.recap()

Type,Number,Size (height*width)
sample,4,2160*2560
ob,5,2160*2560
df,3,2160*2560


# Normalization

In [9]:
o_norm.run_normalization(dict_roi=o_gui.roi_selected)

HBox(children=(Label(value='Normalization', layout=Layout(width='20%')), IntProgress(value=0, max=4)))

# Export 

In [10]:
o_norm.select_export_folder()

HTML(value='Please wait...')

VBox(children=(Label(value='Select Output Folder', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-16259', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(Select(description='Select', layout=Layout(width='750px'), options=(' .', ' ..', ' .DS_Store                                         |     Mon Feb 12 12:59:05 2018', ' Day_4_CO-8_Uptake                                 |     Directory', ' Day_4_CO-8_Uptake_profiles                        |     Directory', ' Day_4_CO-8_Uptake_profiles.txt                    |     Fri Dec  8 12:58:35 2017', ' Day_4_CO-8_Uptake_profiles_DEBUGGING              |     Directory', ' Day_4_CO-8_Uptake_timestamp_infos.txt             |     Wed Jan 10 15:30:58 2018', ' MS-19                                             |     Directory', ' OB                                                |     Directory', ' df                                                |     Directory', ' from_analysis_computer                            |     Directory', ' ideal_water_intake_images                         |     Directory', ' ideal_water_intake_images_profiles                |     Directory', ' ideal_water_intake_images_profiles.txt            |     Wed Dec 13 10:23:47 2017', ' ideal_water_intake_images_timestamp_infos.txt     |     Wed Dec 13 10:21:31 2017'), value=' .'), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

HTML(value='Please wait...')

VBox(children=(Label(value='Select Output Folder', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Users/j35/Desktop', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(Select(description='Select', layout=Layout(width='750px'), options=(' .', ' ..', ' .DS_Store                                   |     Mon Feb 12 12:38:18 2018', ' CI_2018_FO_etal_1771127_jc.docx             |     Mon Nov 27 11:35:09 2017', ' IPTS-18813-logbook.pdf                      |     Tue Jan 23 16:28:33 2018', ' Imaging_Schedule_2018A_Post_Bac.xlsx        |     Thu Dec 14 12:25:45 2017', ' Invoice from JetBrains INVJBA570077.eml     |     Thu Feb  8 09:47:34 2018', ' MS-19_normalized                            |     Directory'), value=' .'), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

In [11]:
o_norm.o_norm.normalization(notebook=True)

In [12]:
o_norm.normalized_data_array

[array([[0.98551437, 0.98508357, 1.07994249, ..., 1.06855619, 1.17192829,
         1.25286153],
        [1.0126989 , 1.01651243, 1.13732011, ..., 1.05063437, 1.25309503,
         1.04298205],
        [0.982886  , 1.19302911, 0.98624464, ..., 0.88197632, 1.05033675,
         1.11780549],
        ...,
        [1.04532568, 1.1127232 , 1.14186216, ..., 1.04838862, 1.14255743,
         1.05840116],
        [1.01752174, 1.09142664, 1.06185375, ..., 1.06925991, 1.04309882,
         1.10478923],
        [1.07859813, 1.05793908, 1.14692506, ..., 1.00587952, 1.08550496,
         1.15414635]]),
 array([[1.03679426, 0.93270694, 1.05632462, ..., 0.88119407, 0.98729717,
         0.93944411],
        [1.03017369, 0.99594177, 1.0125596 , ..., 0.97050968, 0.97074679,
         0.92580653],
        [0.92246282, 1.08744696, 1.07429041, ..., 0.85114114, 0.92172302,
         0.94261262],
        ...,
        [1.00934492, 0.98398528, 1.00767346, ..., 1.0023779 , 1.04210858,
         1.02197033],
        [1.0

In [15]:
o_norm.export()

IntProgress(value=0, max=4)